In [1]:
import pandas as pd
import numpy as np

from os import listdir

In [ ]:
#file_list = set(listdir("C:/Users/paul8/Documents/Uni/7. Thesis/Thesis/paul"))
db = pd.read_csv("C:/Users/paul8/Downloads/DB.csv",delimiter=";")
#file_list_db = set(db.File)
#len(file_list.intersection(file_list_db))
def make_date(s):
    s = str(s)
    if len(s) == 5:
        s = "0" + s
    return s[:2] + "/" + s[2:4] + "/20" + s[4:]
db["Invoice Date"] = db["Invoice Date"].map(make_date)
db.head()
        


,File,Vendor,Invoice Number,Amount,Invoice Date
0,AH_Siedle_Reifenwechsel_24042020.pdf,Autohaus Siedle,110024940,"79,85",04/24/2020
1,ALU_RAHMEN_Rechnung_27112023.pdf,Alurahmen24,1164/11/2023,"23,7",11/27/2023
2,Amazon_GaN_Charger_13102022.pdf,Amazon Service Europe,NaN,"47,99",10/13/2022
3,Amazon_USB_Flash_Drive_24102022.pdf,Amazon EU Sarl,NaN,"20,39",10/24/2022
4,Aral_Center_Hildesheim_05032020.pdf,Aral Center Hildesheim,4605/009/00003,"68,84",03/05/2020


In [ ]:
#file_list_db == file_list

True

In [65]:
data = pd.DataFrame(columns=["File","Extracted Vendor","Extracted Invoice Number","Extracted Amount","Extracted Invoice Date"])
folder = "./Results/"
files = listdir(folder)
for file in files:
    path = folder + file
    data_new = pd.read_csv(path, sep=';', header=None)
    data_new.columns = ["File","Extracted Vendor","Extracted Invoice Number","Extracted Amount","Extracted Invoice Date"]
    #print(data_new.dtypes)
    data = data._append(data_new,ignore_index=True)
data.head()

,File,Extracted Vendor,Extracted Invoice Number,Extracted Amount,Extracted Invoice Date
0,B24_Rechnung 50046676.pdf,BIKE24 Bike24 GmbH Bike24 GmbH,50046676,NaN,04/24/2021 00:00:000


In [ ]:
data["Extracted Invoice Date"] = data["Extracted Invoice Date"].map(lambda x:x.split(" ")[0],na_action="ignore")
data["Extracted Invoice Number"] = data["Extracted Invoice Number"].map(lambda x:str(x),na_action="ignore")
data["Extracted Amount"] = data["Extracted Amount"].map(lambda x:str(x),na_action="ignore")

data.head()


,File,Extracted Vendor,Extracted Invoice Number,Extracted Amount,Extracted Invoice Date
0,B24_Rechnung 50046676.pdf,BIKE24 Bike24 GmbH Bike24 GmbH,50046676,NaN,04/24/2021


In [ ]:
results = data.set_index("File").join(db.set_index("File"),how="inner")
results.head()

,Extracted Vendor,Extracted Invoice Number,Extracted Amount,Extracted Invoice Date,Vendor,Invoice Number,Amount,Invoice Date
File,,,,,,,,
B24_Rechnung 50046676.pdf,BIKE24 Bike24 GmbH Bike24 GmbH,50046676,NaN,04/24/2021,Bike24,50046676,"340,53",04/24/2021


In [ ]:
results["Correct Vendor"] = results["Extracted Vendor"] == results["Vendor"] 
for file,ext_ven,ven in zip(results.index, results["Extracted Vendor"], results["Vendor"]):
    results.loc[file,"Correct Vendor"] = ext_ven in ven or ven in ext_ven
results["Correct Invoice Number"] = results["Extracted Invoice Number"] == results["Invoice Number"]
results["Correct Amount"] = results["Extracted Amount"] == results["Amount"]
results["Correct Invoice Date"] = results["Extracted Invoice Date"] == results["Invoice Date"]
results.head()

,Extracted Vendor,Extracted Invoice Number,Extracted Amount,Extracted Invoice Date,Vendor,Invoice Number,Amount,Invoice Date,Correct Vendor,Correct Invoice Number,Correct Amount,Correct Invoice Date
File,,,,,,,,,,,,
B24_Rechnung 50046676.pdf,BIKE24 Bike24 GmbH Bike24 GmbH,50046676,NaN,04/24/2021,Bike24,50046676,"340,53",04/24/2021,True,True,False,True


In [72]:
n = len(results.index)
vendor_acc = (results["Correct Vendor"].sum() / n) * 100
number_acc = (results["Correct Invoice Number"].sum() / n) * 100
amount_acc = (results["Correct Amount"].sum() / n) * 100
date_acc = (results["Correct Invoice Date"].sum() / n) * 100
print(f"Vendor Accuracy: {vendor_acc} %")
print(f"Invoice Number Accuracy: {number_acc} %")
print(f"Amount Accuracy: {amount_acc} %")
print(f"Invoice Date Accuracy: {date_acc} %")


Vendor Accuracy: 100.0 %
Invoice Number Accuracy: 100.0 %
Amount Accuracy: 0.0 %
Invoice Date Accuracy: 100.0 %
